# Example for importing data from the open-data platform of the Valencia town council

In [8]:
import numpy
import pandas

## The available stations are the ones include in the list 'stations'.

### 'station_data' is created as a dictionary for containing the loaded data-frame objects.

In [9]:
stations=[ '1A', '3A', '4A', '5A', '6A' ]

station_data = dict()

for station in stations:
    d = pandas.read_csv( 'http://mapas.valencia.es/WebsMunicipales/uploads/atmosferica/'+station+'.csv', sep=';' )
    station_data[station] = d

### Preparation of the plotting environment. In this case to show the plots in the notebook. 

In [10]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure

output_notebook()

Loading BokehJS ...

### Prepare 'keys' as a list with all the available measures.

#### Not all the stations have all the measures.

In [11]:
keys = list()
for df in station_data.values():
    for key in df.keys():
        if key not in keys:
            keys.append( key )
#keys = numpy.unique( keys )
if 'Fecha' in keys:
    keys.remove( 'Fecha' )
print( keys )

['R.Sol.(W/m�)', 'PM2,5(�g/m�)', 'PM1(�g/m�)', 'Xileno(�g/m�)', 'SO2(�g/m�)', 'CO(mg/m�)', 'NO(�g/m�)', 'NO2(�g/m�)', 'PM10(�g/m�)', 'NOx(�g/m�)', 'Ozono(�g/m�)', 'Tolueno(�g/m�)', 'Benceno(�g/m�)', 'Ruido(dBA)', 'Veloc.(m/s)', 'Temp.(�C)', 'H.Rel.(% H.R.)', 'Pres.(mb)', 'PM2.5(�g/m�)', 'Ni(ng/m�)', 'As(ng/m�)', 'Pb(�g/m�)', 'BaP(ng/m�)', 'Cd(ng/m�)']


### A conversion of dates stored as strings in the Spanish convention 'dd/mm/yyyy' is needed in order to represent data with time in the x-axis.

In [12]:
from pandas import Timestamp

#help(Timestamp)
def dd_mm_yyyy_to_yyyy_mm_dd(s):
    d,m,y = [int(v) for v in s.split('/')]
    #y,m,d = [int(v) for v in s.split('-')]
    #return Timestamp( y, m, d )
    #return Timestamp( ts_input=s, tz='Europe/Madrid' )
    return Timestamp( ts_input="%d-%d-%d" % (y,m,d) )

for station in stations:
    df = station_data[station]
    x = df['Fecha'].copy()
    for i in range(len(x)):
        #print(type(x[i]))
        x[i] = dd_mm_yyyy_to_yyyy_mm_dd(x[i])
    df['Fecha'] = x

### A palette of colors should be used to represent the data of each station in a different color.

#### Not all the installations of Bokeh have available the same set of palettes, so you might need to change the code of the next cell.

In [13]:
import bokeh.palettes

print( bokeh.palettes.brewer.keys() )
print( bokeh.palettes.brewer['PuBuGn'] )

colors = bokeh.palettes.brewer[ 'PuBuGn' ][ 6 ]
#colors += brewer[ 'Accent'][8]
colors = ['red', 'blue', 'green', 'magenta', 'orange', 'black' ]
#print(colors)

dict_keys(['OrRd', 'Oranges', 'RdYlBu', 'Reds', 'YlOrBr', 'RdBu', 'YlOrRd', 'YlGn', 'BuPu', 'RdYlGn', 'Greens', 'Blues', 'PuBuGn', 'Spectral', 'Purples', 'RdPu', 'PRGn', 'PuRd', 'RdGy', 'PiYG', 'Greys', 'GnBu', 'BuGn', 'PuOr', 'YlGnBu', 'PuBu', 'BrBG'])
{3: ['#1c9099', '#a6bddb', '#ece2f0'], 4: ['#02818a', '#67a9cf', '#bdc9e1', '#f6eff7'], 5: ['#016c59', '#1c9099', '#67a9cf', '#bdc9e1', '#f6eff7'], 6: ['#016c59', '#1c9099', '#67a9cf', '#a6bddb', '#d0d1e6', '#f6eff7'], 7: ['#016450', '#02818a', '#3690c0', '#67a9cf', '#a6bddb', '#d0d1e6', '#f6eff7'], 8: ['#016450', '#02818a', '#3690c0', '#67a9cf', '#a6bddb', '#d0d1e6', '#ece2f0', '#fff7fb'], 9: ['#014636', '#016c59', '#02818a', '#3690c0', '#67a9cf', '#a6bddb', '#d0d1e6', '#ece2f0', '#fff7fb']}


### Every time the following cell is executed a different key (measure) is chosen to be plotted.

In [27]:
key = keys[ numpy.random.randint(len(keys)) ]
#print(key)

fig = figure( title='VLC Pollution', plot_width=980, plot_height=700,
              x_axis_type='datetime',
              x_axis_label='Time', y_axis_label=key )

color_index=0
for station in stations:
    df = station_data[station]
    if key in df.keys():
        x = df['Fecha']
        y = df[key]
        fig.line( x, y, legend=station, color=colors[color_index], line_width=2 )
        color_index += 1
        #break

show(fig)

ValueError: Out of range float values are not JSON compliant